In [10]:
from luminoso_api import LuminosoClient
from pack64 import unpack64
from tableau_export import pull_lumi_data

import numpy as np

In [2]:
client, docs, topics, terms, subsets, drivers, skt, themes = pull_lumi_data('d53m338v', 'pr3n28bw')

Extracting Lumi data...


	Waiting for recalculation (120sec)

Done recalculating. Training...
Done training.


In [6]:
print(len(unpack64(topics[0]['vector'])))

150


In [8]:
print(len(unpack64(docs[0]['vector'])))

150


In [11]:
np.dot(unpack64(topics[0]['vector']), unpack64(docs[0]['vector']))

0.18078466

In [15]:
len(terms)

1000

In [14]:
topics[0]

{'_id': 'f72399a7-fd49-44c7-92aa-a475b66f20ad',
 'color': '#00aaff',
 'name': 'drinks',
 'text': 'drinks',
 'vector': 'W-lt_v-JFk9BCATj5mCBLN6w6B2OMqy30-AFf5Wk_gI3WXIR868-HAL-UVK0d8QdBZoC0PyTwzbVRyQ3739QqEefA_2A1L6ovBRuASA4s5C-g5nR2CM9ky8hE_iwPSA5Kw9o38P6HP26wcA4SBEBDT7E5zO2C8RuA6J_tA-AADve9qTEZD5KWKgHArJBDZHky2GSGtI8-43sS8ja4ek_uD53K_7j_e5Ep3D4V-_D3lD5QOH4tDRbD3_E2D-A1_lY5NQH89CVJBWVA2_AZN_4T6XGGPbBTVD0j-5O_rc8dhBev-uz_1LDEz-D_EEIBQB-_CCdx63d7yS-vZ86--L0BSZDgIE0A9eKGId-Ku-Xb6hvEkSBjO64ABmJ6NZDBj_JGBSH6cU_FBA8FAk84e8-eF9KK6ZzAee-MA9EEDXmAzYHj3-t48rq80dDHzAZtAE6BfT'}

In [18]:
%run tableau_export.py d53m338v pr3n28bw -t 150

Extracting Lumi data...
Creating doc table...


/usr/local/lib/python3.6/site-packages/se_code-0.1-py3.6.egg/se_code/fuzzy_logic.py:38: RuntimeWarning: invalid value encountered in double_scalars


Writing to file doc_table.csv.
Writing to file xref_table.csv.
Writing to file doc_term_table.csv.
Writing to file doc_topic_table.csv.
Creating themes table...
Writing to file themes_table.csv.
Creating subset key terms table...
Writing to file skt_table.csv.
Writing to file drivers_table.csv.
Writing to file trends_table.csv.
Writing to file trendingterms_table.csv.
